# valueguard-area-information-formated-excel

### Install valueguard lib

In [ ]:
%%capture --no-display
!pip3 install --upgrade --force-reinstall  git+https://github.com/Valueguard-Index-Sweden/valueguard-python-client#egg=valueguard

### Imports

In [30]:
import valueguard 
import requests
import pandas as pd
import numpy as np
from datetime import datetime

### Settings

In [2]:
pd.set_option('display.max_colwidth', None)

### Authenticate [ API ]

In [ ]:
# Create object
vgClient = valueguard.Client()

# Generate a access token
vgClient.authenticate("username","password")

### Generate EXCEL file

In [ ]:
%%time
print("Area")
area_df = pd.DataFrame(vgClient.area()['areas'])

# Get latest data
current_date = datetime.now().strftime("%Y-%m-%d")
print("Data valid at: ", current_date)

print("Area information")
area_information_df = pd.DataFrame(vgClient.area_information(search_criteria={
                                                                              "valid_at": current_date
                                                                             })['area_information'])

In [5]:
%%time
fields = ["avg_recounted_price_n_best", "avg_recounted_price_n", "avg_recounted_psqm_n_best", "avg_recounted_psqm_n", "number_of_sales_n"]
excel_rows_description = [
    {'rooms': "1", 'category': "Bostadsrätt", "suffix": "_oneroom"},
    {'rooms': "2", 'category': "Bostadsrätt", "suffix": "_tworoom"},
    {'rooms': "3", 'category': "Bostadsrätt", "suffix": "_threeroom"},
    {'rooms': "4", 'category': "Bostadsrätt", "suffix": "_fourroom"},
    {'rooms': "5", 'category': "Bostadsrätt", "suffix": "_fiveroom_plus"},
    {'rooms': None, 'category': "Bostadsrätt", "suffix": "_apt"},
    {'rooms': None, 'category': "Småhus", "suffix": "_house"}
]

excel_result_raw = []

for index, area_row in area_df.iterrows():
    area_id = area_row['id']
    area_name = area_row['name']
    
    for row_info in excel_rows_description:
        row = {'deso': area_name, 'rum': row_info['rooms'], 'kategori': row_info['category']}
        
        tmp_df = area_information_df[area_information_df['area_id'] == area_id]
        for field in fields:
            field_rows = tmp_df[tmp_df['field_name'] == field + row_info['suffix']]
            
            
            field_value = field_rows.iloc[0]['value'] if len(field_rows) > 0 else None
            row[field] = field_value
        
        excel_result_raw.append(row)


CPU times: user 58.6 s, sys: 12.3 ms, total: 58.6 s
Wall time: 58.6 s


In [6]:
excel_result_df = pd.DataFrame(excel_result_raw)
excel_result_df['medelpris'] = np.where(excel_result_df['avg_recounted_price_n_best'].isnull(), excel_result_df['avg_recounted_price_n'], excel_result_df['avg_recounted_price_n_best'])
excel_result_df['medelkvmpris'] = np.where(excel_result_df['avg_recounted_psqm_n_best'].isnull(), excel_result_df['avg_recounted_psqm_n'], excel_result_df['avg_recounted_psqm_n_best'])
excel_result_df['antal'] = excel_result_df['number_of_sales_n']

In [7]:
excel_final_df = excel_result_df[['deso','rum','kategori','medelpris','medelkvmpris','antal']][(excel_result_df['medelpris'].notnull()) | (excel_result_df['medelkvmpris'].notnull())]

In [12]:
#excel_final_df[:10]

In [9]:
excel_final_df.to_excel("valueguard_area_information_formated.xlsx", index=False)